<a href="https://colab.research.google.com/github/AMAYadav/finance_models/blob/main/Ima_plat_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-networkdays --quiet
!pip install py_vollib_vectorized --quiet

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import networkdays
import datetime
import py_vollib.black_scholes_merton.implied_volatility
import py_vollib_vectorized

In [ ]:
position='call'
k=1200
r=0.05


In [ ]:
data_raw=pd.read_excel('/content/5819-PnL Attribution and Backtesting (PLAT).xlsm',sheet_name='data')
data_raw.iloc[1:,:].rename(columns=data_raw.iloc[1], inplace = False)

,Date,NIFTY_SPOT,Option,Time to Maturity,BSM_vol,d1,d2,BSM_value,delta,gamma,vega,theta,APL ($),HPL ($),RTPL ($),1-day VaR (99%)
1,Date,NIFTY_SPOT,Option,Time to Maturity,BSM_vol,d1,d2,BSM_value,delta,gamma,vega,theta,APL ($),HPL ($),RTPL ($),1-day VaR (99%)
2,2020-02-07 00:00:00,12098.34961,915,0.94,0.116048,0.546535,0.434023,915.0,0.707651,0.000252,4030.307058,-631.10089,0,0,0,0
3,2020-02-10 00:00:00,12031.5,869.95,0.936,0.117067,0.492987,0.379728,869.95,0.688989,0.000259,4112.378657,-628.152965,-45.05,-45.156212,-47.306185,-146.179305
4,2020-02-11 00:00:00,12107.90039,942,0.932,0.122323,0.529457,0.411366,942,0.701756,0.000243,4053.358586,-643.737833,72.05,72.497837,52.639041,-142.781825
5,2020-02-12 00:00:00,12201.2002,993.55,0.928,0.119166,0.606437,0.491641,993.55,0.727888,0.000237,3901.460415,-644.875151,51.55,51.157862,65.473693,-152.921769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
592,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
593,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
594,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
595,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
expiry=pd.read_excel('/content/5819-PnL Attribution and Backtesting (PLAT).xlsm',sheet_name='data').iloc[:1,2:3]
expiry['Unnamed: 2']=pd.to_datetime(expiry['Unnamed: 2'])
expiry

,Unnamed: 2
0,2020-12-31


In [ ]:
data=data_raw.iloc[2:,].rename(columns=data_raw.iloc[1], inplace = False)
data_new=data.iloc[:,:12]
data_new['Date']=pd.to_datetime(data_new['Date'],errors='coerce').dropna(inplace=False)
data_new

<ipython-input-7-221a854b4b5b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_new['Date']=pd.to_datetime(data_new['Date'],errors='coerce').dropna(inplace=False)


,Date,NIFTY_SPOT,Option,Time to Maturity,BSM_vol,d1,d2,BSM_value,delta,gamma,vega,theta
2,2020-02-07,12098.34961,915,0.94,0.116048,0.546535,0.434023,915.0,0.707651,0.000252,4030.307058,-631.10089
3,2020-02-10,12031.5,869.95,0.936,0.117067,0.492987,0.379728,869.95,0.688989,0.000259,4112.378657,-628.152965
4,2020-02-11,12107.90039,942,0.932,0.122323,0.529457,0.411366,942,0.701756,0.000243,4053.358586,-643.737833
5,2020-02-12,12201.2002,993.55,0.928,0.119166,0.606437,0.491641,993.55,0.727888,0.000237,3901.460415,-644.875151
6,2020-02-13,12174.65039,961.45,0.924,0.116542,0.5974,0.485375,961.45,0.72488,0.000245,3905.759684,-639.497294
...,...,...,...,...,...,...,...,...,...,...,...,...
592,NaT,0,0,0,0,0,0,0,0,0,0,0
593,NaT,0,0,0,0,0,0,0,0,0,0,0
594,NaT,0,0,0,0,0,0,0,0,0,0,0
595,NaT,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
real_data=data_new.dropna()
real_data

,Date,NIFTY_SPOT,Option,Time to Maturity,BSM_vol,d1,d2,BSM_value,delta,gamma,vega,theta
2,2020-02-07,12098.34961,915,0.94,0.116048,0.546535,0.434023,915.0,0.707651,0.000252,4030.307058,-631.10089
3,2020-02-10,12031.5,869.95,0.936,0.117067,0.492987,0.379728,869.95,0.688989,0.000259,4112.378657,-628.152965
4,2020-02-11,12107.90039,942,0.932,0.122323,0.529457,0.411366,942,0.701756,0.000243,4053.358586,-643.737833
5,2020-02-12,12201.2002,993.55,0.928,0.119166,0.606437,0.491641,993.55,0.727888,0.000237,3901.460415,-644.875151
6,2020-02-13,12174.65039,961.45,0.924,0.116542,0.5974,0.485375,961.45,0.72488,0.000245,3905.759684,-639.497294
...,...,...,...,...,...,...,...,...,...,...,...,...
219,2020-12-23,13601.09961,1616.1,0.028,0.06,12.619064,12.609024,1617.887855,1,0.0,0.0,-599.160588
220,2020-12-24,13749.25,1764.4,0.024,0.05,17.726338,17.718593,1763.641363,1,0.0,0.0,-599.280432
221,2020-12-28,13873.2002,1896.4,0.016,0.05,23.064434,23.058109,1882.796361,1,0.0,0.0,-599.520192
222,2020-12-29,13932.59961,1940.35,0.012,0.04,34.217675,34.213293,1939.79745,1,0.0,0.0,-599.640108


In [ ]:

# price – The price of the option.

# F – The price of the underlying asset.

# K – The strike price.

# t – The annualized time to expiration. Must be positive. For small TTEs, use a small value (1e-3).

# r – The Interest Free Rate.

# flag – For each contract, this should be specified as c for a call option and p for a put option.

# on_error – Either “raise”, “warn” or “ignore”.

# return_as – To return as a pd.Series object, use “series”. To return as a pd.DataFrame object, use “dataframe”. Any other value will return a numpy.array object.

# dtype – Data type.

# kwargs – Other keyword arguments are ignored.

F=af['NIFTY_SPOT']
price=af['Option']
K=1200
t=af['Time to Maturity']
r=0.02
flag=['c']
on_error='ignore'

In [ ]:
py_vollib.black.implied_volatility.implied_volatility(price, F, K, r, t, flag)

In [ ]:
# real_data.info()
a=pd.DataFrame(np.busday_count(real_data.index.values.astype('M8[D]'),expiry.values.astype('M8[D]')).transpose()/250,columns=['dates'])

In [ ]:
import math
import scipy.stats as stats
from scipy.optimize import newton

def black_scholes_option_price(S, K, T, r, sigma, option_type='call'):
    """
    Calculate the Black-Scholes option price for a call or put option.

    S: Current stock price
    K: Strike price
    T: Time to expiration (in years)
    r: Risk-free interest rate (annualized)
    sigma: Implied volatility
    option_type: 'call' or 'put'

    Returns the option price.
    """
    d1 = (math.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)

    if option_type == 'call':
        option_price = S * stats.norm.cdf(d1) - K * math.exp(-r * T) * stats.norm.cdf(d2)
    elif option_type == 'put':
        option_price = K * math.exp(-r * T) * stats.norm.cdf(-d2) - S * stats.norm.cdf(-d1)
    else:
        raise ValueError("Invalid option_type. Use 'call' or 'put'.")

    return option_price

def implied_volatility(option_price, S, K, T, r, option_type='call'):
    """
    Calculate the implied volatility of an option using the Newton-Raphson method.

    option_price: Observed market price of the option
    S: Current stock price
    K: Strike price
    T: Time to expiration (in years)
    r: Risk-free interest rate (annualized)
    option_type: 'call' or 'put'

    Returns the implied volatility.
    """

    def f(sigma):
        return black_scholes_option_price(S, K, T, r, sigma, option_type) - option_price

    def f_prime(sigma):
        d1 = (math.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * math.sqrt(T))
        if option_type == 'call':
            return S * math.sqrt(T) * stats.norm.pdf(d1)
        elif option_type == 'put':
            return S * math.sqrt(T) * stats.norm.pdf(d1) * (-1)

    initial_guess = 0.2  # Initial guess for implied volatility
    implied_vol = newton(f, initial_guess, fprime=f_prime)

    return implied_vol

# Example usage:
S = 100  # Current stock price
K = 100  # Strike price
T = 1    # Time to expiration (in years)
r = 0.05 # Risk-free interest rate (annualized)
option_price = 10  # Observed market price of the option
option_type = 'call'

iv = implied_volatility(option_price, S, K, T, r, option_type)
print(f"Implied Volatility: {iv:.4f}")


Implied Volatility: 0.1880


In [ ]:
real_data.set_index('Date',inplace=True)

In [ ]:
real_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222 entries, 2020-02-07 to 2020-12-30
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   NIFTY_SPOT        222 non-null    object
 1   Option            222 non-null    object
 2   Time to Maturity  222 non-null    object
 3   BSM_vol           222 non-null    object
 4   d1                222 non-null    object
 5   d2                222 non-null    object
 6   BSM_value         222 non-null    object
 7   delta             222 non-null    object
 8   gamma             222 non-null    object
 9   vega              222 non-null    object
 10  theta             222 non-null    object
dtypes: object(11)
memory usage: 20.8+ KB


In [ ]:
def implied_vol(S,K,T,r,c):
  from scipy import log,exp,sqrt,stats

  for i in range(200):
    sigma=0.04*(i+1)

    d1= (log(S/K)+(r+sigma*sigma/2)*T)/(sigma*sqrt(T))
    d2=d1-sigma*sqrt(T)
    diff= c-(S*stats.norm.cdf(d1)-X*exp(-r*T)*stats*norm*cdf(d2))
    if abs(diff) <= 0.01:
      return sigma,diff

In [ ]:
af=real_data.iloc[:,0:4]
implied_vol(S=af['NIFTY_SPOT'],K=k,T=af['Time to Maturity'],r=r,c=af['Option'])

In [ ]:
real_data.iloc[:,0:6]

from scipy import log,exp,sqrt,stats
sigma=0.05
S=af['NIFTY_SPOT'][0]
K=k
T=af['Time to Maturity'][0]
r=r
c=af['Option']

d1= (log(S/K)+(r+sigma*sigma/2)*T)/(sigma*sqrt(T))
d2=d1-sigma*sqrt(T)
d1
d2

<ipython-input-95-e88b118677cd>:11: DeprecationWarning: scipy.log is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.log instead
  d1= (log(S/K)+(r+sigma*sigma/2)*T)/(sigma*sqrt(T))
<ipython-input-95-e88b118677cd>:11: DeprecationWarning: scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead
  d1= (log(S/K)+(r+sigma*sigma/2)*T)/(sigma*sqrt(T))
<ipython-input-95-e88b118677cd>:12: DeprecationWarning: scipy.sqrt is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.sqrt instead
  d2=d1-sigma*sqrt(T)


48.030657120975064

In [ ]:
###main work


real_data['APL']=real_data['Option']-real_data['Option'].shift()


real_data['HPL']=real_data['delta']*(real_data['NIFTY_SPOT']-real_data['NIFTY_SPOT'].shift())+ 0.5*real_data['gamma']*(real_data['NIFTY_SPOT']-real_data['NIFTY_SPOT'].shift())**2+real_data['vega']*(real_data['BSM_vol']-real_data['BSM_vol'].shift())+real_data['theta']*(real_data['Time to Maturity']-real_data['Time to Maturity'].shift())

real_data['RTPL']=real_data['delta']*(real_data['NIFTY_SPOT']-real_data['NIFTY_SPOT'].shift())
real_data

<ipython-input-103-a38ac0c9d597>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data['APL']=real_data['Option']-real_data['Option'].shift()
<ipython-input-103-a38ac0c9d597>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data['HPL']=real_data['delta']*(real_data['NIFTY_SPOT']-real_data['NIFTY_SPOT'].shift())+ 0.5*real_data['gamma']*(real_data['NIFTY_SPOT']-real_data['NIFTY_SPOT'].shift())**2+real_data['vega']*(real_data['BSM_vol']-real_data['BSM_vol'].shift())+real_data['theta']*(real_data['Tim

,NIFTY_SPOT,Option,Time to Maturity,BSM_vol,d1,d2,BSM_value,delta,gamma,vega,theta,APL,RTPL,HPL
Date,,,,,,,,,,,,,,
2020-02-07,12098.34961,915,0.94,0.116048,0.546535,0.434023,915.0,0.707651,0.000252,4030.307058,-631.10089,NaN,NaN,NaN
2020-02-10,12031.5,869.95,0.936,0.117067,0.492987,0.379728,869.95,0.688989,0.000259,4112.378657,-628.152965,-45.05,-46.058657,-38.772673
2020-02-11,12107.90039,942,0.932,0.122323,0.529457,0.411366,942,0.701756,0.000243,4053.358586,-643.737833,72.05,53.614425,78.201714
2020-02-12,12201.2002,993.55,0.928,0.119166,0.606437,0.491641,993.55,0.727888,0.000237,3901.460415,-644.875151,51.55,67.911792,59.205852
2020-02-13,12174.65039,961.45,0.924,0.116542,0.5974,0.485375,961.45,0.72488,0.000245,3905.759684,-639.497294,-32.1,-19.245424,-26.85248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,13601.09961,1616.1,0.028,0.06,12.619064,12.609024,1617.887855,1,0.0,0.0,-599.160588,120.55,134.7998,137.196442
2020-12-24,13749.25,1764.4,0.024,0.05,17.726338,17.718593,1763.641363,1,0.0,0.0,-599.280432,148.3,148.15039,150.547512
2020-12-28,13873.2002,1896.4,0.016,0.05,23.064434,23.058109,1882.796361,1,0.0,0.0,-599.520192,132.0,123.9502,128.746362


In [ ]:
real_data.iloc[:,-3:]

,APL,RTPL,HPL
Date,,,
2020-02-07,NaN,NaN,NaN
2020-02-10,-45.05,-46.058657,-38.772673
2020-02-11,72.05,53.614425,78.201714
2020-02-12,51.55,67.911792,59.205852
2020-02-13,-32.1,-19.245424,-26.85248
...,...,...,...
2020-12-23,120.55,134.7998,137.196442
2020-12-24,148.3,148.15039,150.547512
2020-12-28,132.0,123.9502,128.746362


In [ ]:
# https://libraries.io/pypi/var#examples

!pip install var==0.0.5
!pip install arch

In [ ]:
from var import VaR
var = VaR(real_data[['NIFTY_SPOT']],weights =np.array([0.1]), alpha=0.05)

In [ ]:
from scipy import stats
real_data['VaR']=((real_data['delta']*real_data['NIFTY_SPOT']*real_data['BSM_vol']*stats.norm.cdf(.99))/math.sqrt(250)).shift(1)
real_data.head()

In [ ]:
real_data

,NIFTY_SPOT,Option,Time to Maturity,BSM_vol,d1,d2,BSM_value,delta,gamma,vega,theta,APL,RTPL,HPL,VaR
Date,,,,,,,,,,,,,,,
2020-02-07,12098.34961,915,0.94,0.116048,0.546535,0.434023,915.0,0.707651,0.000252,4030.307058,-631.10089,NaN,NaN,NaN,None
2020-02-10,12031.5,869.95,0.936,0.117067,0.492987,0.379728,869.95,0.688989,0.000259,4112.378657,-628.152965,-45.05,-46.058657,-38.772673,52.714262
2020-02-11,12107.90039,942,0.932,0.122323,0.529457,0.411366,942,0.701756,0.000243,4053.358586,-643.737833,72.05,53.614425,78.201714,51.489084
2020-02-12,12201.2002,993.55,0.928,0.119166,0.606437,0.491641,993.55,0.727888,0.000237,3901.460415,-644.875151,51.55,67.911792,59.205852,55.145687
2020-02-13,12174.65039,961.45,0.924,0.116542,0.5974,0.485375,961.45,0.72488,0.000245,3905.759684,-639.497294,-32.1,-19.245424,-26.85248,56.152346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,13601.09961,1616.1,0.028,0.06,12.619064,12.609024,1617.887855,1,0.0,0.0,-599.160588,120.55,134.7998,137.196442,71.448838
2020-12-24,13749.25,1764.4,0.024,0.05,17.726338,17.718593,1763.641363,1,0.0,0.0,-599.280432,148.3,148.15039,150.547512,43.298431
2020-12-28,13873.2002,1896.4,0.016,0.05,23.064434,23.058109,1882.796361,1,0.0,0.0,-599.520192,132.0,123.9502,128.746362,36.475051


In [ ]:
real_data['APL_exception']=0
real_data

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real_data['APL_exception']=0



,NIFTY_SPOT,Option,Time to Maturity,BSM_vol,d1,d2,BSM_value,...,vega,theta,APL,RTPL,HPL,VaR,APL_exception
Date,,,,,,,,,,,,,,,
2020-02-07,12098.34961,915,0.94,0.116048,0.546535,0.434023,915.0,...,4030.307058,-631.10089,NaN,NaN,NaN,None,0
2020-02-10,12031.5,869.95,0.936,0.117067,0.492987,0.379728,869.95,...,4112.378657,-628.152965,-45.05,-46.058657,-38.772673,52.714262,0
2020-02-11,12107.90039,942,0.932,0.122323,0.529457,0.411366,942,...,4053.358586,-643.737833,72.05,53.614425,78.201714,51.489084,0
2020-02-12,12201.2002,993.55,0.928,0.119166,0.606437,0.491641,993.55,...,3901.460415,-644.875151,51.55,67.911792,59.205852,55.145687,0
2020-02-13,12174.65039,961.45,0.924,0.116542,0.5974,0.485375,961.45,...,3905.759684,-639.497294,-32.1,-19.245424,-26.85248,56.152346,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,13601.09961,1616.1,0.028,0.06,12.619064,12.609024,1617.887855,...,0.0,-599.160588,120.55,134.7998,137.196442,71.448838,0
2020-12-24,13749.25,1764.4,0.024,0.05,17.726338,17.718593,1763.641363,...,0.0,-599.280432,148.3,148.15039,150.547512,43.298431,0
2020-12-28,13873.2002,1896.4,0.016,0.05,23.064434,23.058109,1882.796361,...,0.0,-599.520192,132.0,123.9502,128.746362,36.475051,0


In [ ]:
real_data['APL_exception']=real_data['APL_exception'].where(real_data.APL < real_data.VaR, 1).shift(1)

In [ ]:
real_data['HPL_exception']=0
real_data['HPL_exception']=real_data['HPL_exception'].where(real_data.HPL < real_data.VaR, 1).shift(1)

In [ ]:
real_data.APL_exception.sum()
real_data.HPL_exception.sum()

63.0

In [ ]:
total_exceptions=max(real_data.APL_exception.sum(),real_data.HPL_exception.sum())
total_exceptions

##designate the range

63.0

In [ ]:
spearman_correl=stats.spearmanr(real_data.APL.dropna(),real_data.HPL.dropna()).statistic*100
spearman_correl

97.8997869020576

In [ ]:
np.max(real_data['Option']-900,0)

1079

In [ ]:
pnl=real_data['Option'].where(real_data['Option'] < k,0)
max(pnl.values,0)